# Анализ данных

# <a id="0">Содержание</a>

- <a href="#1">Открытие данных и их описание</a>
- <a href="#2">Предобработка данных. Добавление признаков</a>  
    - <a href="#21">Обработка дубликатов</a>
    - <a href="#22">Обработка категориальных и временных признаков</a>
    - <a href="#23">Поиск и обработка аномалий в данных</a>
    - <a href="#24">Добавление новых признаков</a>
- <a href="#3">Разведывательный анализ данных</a>  
- <a href="#4">Выводы</a>  

In [262]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# <a id="1">Открытие данных и их описание</a>

Загрузим предобработанные данные

In [263]:
# Откроем данные
users = pd.read_csv('../src/users.tsv', sep='\t')
history = pd.read_csv('../src/history.tsv', sep='\t')
validate = pd.read_csv('../src/validate.tsv', sep='\t')
validate_answers = pd.read_csv('../src/validate_answers.tsv', sep='\t')

In [264]:
# Посмотрим информацию о датасете
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27769 entries, 0 to 27768
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  27769 non-null  int64
 1   sex      27769 non-null  int64
 2   age      27769 non-null  int64
 3   city_id  27769 non-null  int64
dtypes: int64(4)
memory usage: 867.9 KB


In [265]:
# Рассмотрим первые 5 строк
users.head()

,user_id,sex,age,city_id
0,0,2,19,0
1,1,1,0,1
2,2,2,24,2
3,3,1,20,3
4,4,2,29,4


`users`:
- `user_id` – уникальный идентификатор пользователя
- `sex` – указанный пользователем пол в анкете
- `age` – указанный пользователем в анкете возраст пользователя. 0 – не указан.
- `city_id` - указанный пользователем в анкете город проживания. 0 – не указан.
- `age_categorized` - возрастная категория пользователя

In [266]:
# Посмотрим информацию о датасете
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1147857 entries, 0 to 1147856
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   hour       1147857 non-null  int64  
 1   cpm        1147857 non-null  float64
 2   publisher  1147857 non-null  int64  
 3   user_id    1147857 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 35.0 MB


In [267]:
# Рассмотрим первые 5 строк
history.head()

,hour,cpm,publisher,user_id
0,10,30.00,1,15661
1,8,41.26,1,8444
2,7,360.00,1,15821
3,18,370.00,1,21530
4,8,195.00,2,22148


`history`:
- `hour` – в какой час пользователь видел объявление
- `cpm` - цена показанного рекламного объявления в рекламном аукционе. Это значит, что на данном аукционе это была максимальная ставка. 
- `publisher` - площадка, на который пользователь видел рекламу
- `user_id` - уникальный идентификатор пользователя
- `sex` – указанный пользователем пол в анкете
- `city_id` - указанный пользователем в анкете город проживания. 0 – не указан.
- `age_categorized` - возрастная категория пользователя

In [268]:
# Посмотрим информацию о датасете
validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cpm            1008 non-null   float64
 1   hour_start     1008 non-null   int64  
 2   hour_end       1008 non-null   int64  
 3   publishers     1008 non-null   object 
 4   audience_size  1008 non-null   int64  
 5   user_ids       1008 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 47.4+ KB


In [269]:
# Рассмотрим первые 5 строк
validate.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2..."


`validate`:
- `cpm` - для какой цены объявления нужно сделать прогноз
- `hour_start` - предположительное время запуска рекламного объявления
- `hour_end` - предположительное время остановки рекламного объявления. По итогу прогноз делается для рекламного объявление, которое будет запущено в период времени `[hour_start, hour_end]`
- `publishers` - на каких площадках объявление может быть показано
- `audience_size` - размер аудитории объявления, количество идентификаторов в поле `user_ids`
- `user_ids` – аудитория объявления – список пользователей, кому рекламодатель хочет показать объявление.

В датасете 1008 строк. Проупсков нет

## Сборка датасета

### 1. Агрегация по пользователям

Сгруппируем датасет с агрегацией по пользователям, где посчитаем общее количество просмотренной рекламы, срденюю, медианную, максимальную стоимость рекламы

In [270]:
# Агрегация по пользователям
# Общее количество объявлений, просмотренных пользователем
user_agg = history.groupby('user_id').agg(
    total_ads=('cpm', 'count'),
    avg_cpm=('cpm', 'mean'),
    median_cpm=('cpm', 'median'),
    max_cpm=('cpm', 'max')
).reset_index()

# Посмотрим что получилось
user_agg

,user_id,total_ads,avg_cpm,median_cpm,max_cpm
0,0,2,2053.830000,2053.830,3302.01
1,1,82,124.068049,90.000,1140.00
2,3,8,370.880000,288.130,715.92
3,4,132,44.627955,30.160,255.36
4,5,9,91.055556,42.500,390.00
...,...,...,...,...,...
25531,27764,43,210.409767,170.000,958.11
25532,27765,79,146.901013,110.640,608.48
25533,27766,54,159.045741,123.525,755.00
25534,27767,1,143.880000,143.880,143.88


Сделаем датасет с рапределением просмотров рекламы на 24 часа. В каждом часе проставим сколько раз была просмотрена реклама

In [271]:
# Расчет распределения часов просмотра рекламы
hour_distribution = history.groupby('user_id')['hour'].apply(
    lambda x: np.histogram(x, bins=24, range=(0, 23))[0]
).reset_index(name='hours_distribution')

# Преобразование гистограммы в отдельные колонки для часов
hour_columns = [f'hour_{i}' for i in range(24)]
hour_distribution_df = pd.DataFrame(hour_distribution['hours_distribution'].tolist(), columns=hour_columns)
hour_distribution = pd.concat([hour_distribution['user_id'], hour_distribution_df], axis=1)

# Рассмотрим полученный датасет
hour_distribution

,user_id,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25531,27764,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25532,27765,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25533,27766,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25534,27767,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Теперь объединим обе таблицы

In [272]:
# Объединение user_agg и hour_distribution
user_agg = user_agg.merge(hour_distribution, on='user_id', how='left')
user_agg

,user_id,total_ads,avg_cpm,median_cpm,max_cpm,hour_0,hour_1,hour_2,hour_3,hour_4,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,2,2053.830000,2053.830,3302.01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,82,124.068049,90.000,1140.00,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,8,370.880000,288.130,715.92,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,132,44.627955,30.160,255.36,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
4,5,9,91.055556,42.500,390.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25531,27764,43,210.409767,170.000,958.11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25532,27765,79,146.901013,110.640,608.48,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25533,27766,54,159.045741,123.525,755.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25534,27767,1,143.880000,143.880,143.88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Агрегация по площадкам (publisher)

Теперь сделаем агрегацию по площадкам, где размещается объявление

In [273]:
# # Агрегация по площадкам
publisher_agg = history.groupby('publisher').agg(
    total_ads=('cpm', 'count'),
    avg_cpm=('cpm', 'mean'),
    median_cpm=('cpm', 'median'),
    max_cpm=('cpm', 'max')
).reset_index()

publisher_agg

,publisher,total_ads,avg_cpm,median_cpm,max_cpm
0,1,692535,177.567497,105.020,64282.98
1,2,273037,203.134623,126.520,55909.62
2,3,72124,195.899285,100.150,209053.98
3,4,2286,92.164563,63.125,1805.75
4,5,7263,183.900573,108.920,21655.29
5,6,5935,87.411821,45.000,1026.60
6,7,66134,244.468823,130.000,33147.27
7,8,3625,115.163796,82.450,1011.68
8,9,10165,98.388453,66.930,1575.87
9,10,4232,108.062290,60.000,930.32


### 3. Сессии польхователей

Выведем новый столбец в номером сессии `session_id`.

Из условия задачи:
- Система запоминает все объявления, который видел пользователь в течении сессии и не показывает их, даже если это самые дорогие объявления в аукционе.  Новая сессия начинается после отсутсвия показов рекламы пользователю в течении 6 часов.

In [274]:
# Сортировка данных для вычисления сессий
history = history.sort_values(by=['user_id', 'hour'])

# Вычисление сессий
history['time_diff'] = history.groupby('user_id')['hour'].diff().fillna(0)
history['new_session'] = (history['time_diff'] > 6).astype(int)
history['session_id'] = history.groupby('user_id')['new_session'].cumsum()
history

,hour,cpm,publisher,user_id,time_diff,new_session,session_id
778194,1186,805.65,1,0,0.0,0,0
793167,1217,3302.01,1,0,31.0,1,1
305,8,165.00,1,1,0.0,0,0
1015,9,53.20,1,1,1.0,0,0
3458,9,36.00,1,1,0.0,0,0
...,...,...,...,...,...,...,...
782782,1188,285.00,2,27768,46.0,1,17
790311,1188,205.00,12,27768,0.0,0,17
1059564,1216,210.00,2,27768,28.0,1,18
936546,1410,179.80,2,27768,194.0,1,19


Далее рассмотрим сессии пользователей и тоже сделаем по ним агрециии !!!

In [275]:
# Агрегация сессий
session_stats = history.groupby(['user_id', 'session_id']).agg(
    session_ads=('cpm', 'count')
).reset_index()

# Агрегация сессий по пользователям
session_agg = session_stats.groupby('user_id').agg(
    avg_ads_per_session=('session_ads', 'mean'),
    max_ads_per_session=('session_ads', 'max')
).reset_index()

session_agg

,user_id,avg_ads_per_session,max_ads_per_session
0,0,1.000000,1
1,1,2.157895,7
2,3,1.000000,1
3,4,2.095238,7
4,5,1.285714,2
...,...,...,...
25531,27764,1.482759,3
25532,27765,1.837209,7
25533,27766,1.687500,6
25534,27767,1.000000,1


### 4. Добавление данных пользователей

In [276]:
# Посмотрим количество пользователей по полу
users['sex'].value_counts()

sex
1    14515
2    13224
0       30
Name: count, dtype: int64

Преобразуем в бинарные данные столбец по полу, так как всего 30 пользователей не указали пол. Также разделим пользователей по группам возрастов

In [277]:
# Добавление данных пользователей
users['sex_binary'] = (users['sex'] == 1).astype(int)
users['age_group'] = pd.cut(users['age'], bins=[0, 18, 35, 50, np.inf], labels=['0-18', '19-35', '36-50', '51+'], right=False)
users

,user_id,sex,age,city_id,sex_binary,age_group
0,0,2,19,0,0,19-35
1,1,1,0,1,1,0-18
2,2,2,24,2,0,19-35
3,3,1,20,3,1,19-35
4,4,2,29,4,0,19-35
...,...,...,...,...,...,...
27764,27764,1,38,295,1,36-50
27765,27765,2,30,79,0,19-35
27766,27766,2,21,1953,0,19-35
27767,27767,2,17,0,0,0-18


Также определим топ-10 городов по количеству пользователей, а остальные определим как `other`

In [278]:
# Группировка городов по частоте
city_counts = users['city_id'].value_counts()
top_cities = city_counts.nlargest(10).index
users['city_group'] = users['city_id'].apply(lambda x: x if x in top_cities else 'other')
users

,user_id,sex,age,city_id,sex_binary,age_group,city_group
0,0,2,19,0,0,19-35,0
1,1,1,0,1,1,0-18,other
2,2,2,24,2,0,19-35,other
3,3,1,20,3,1,19-35,3
4,4,2,29,4,0,19-35,other
...,...,...,...,...,...,...,...
27764,27764,1,38,295,1,36-50,other
27765,27765,2,30,79,0,19-35,other
27766,27766,2,21,1953,0,19-35,other
27767,27767,2,17,0,0,0-18,0


### 5. Объединение фичей

Объединим фичи с пользователями и с сессиями и сохраним. Отдельно сохраним датасет с площадками

In [279]:
# Объединение всех фичей
user_features = user_agg.merge(session_agg, on='user_id', how='left')
user_features = user_features.merge(users, on='user_id', how='left')

In [280]:
# Посмотрим полученный датасет
user_features

,user_id,total_ads,avg_cpm,median_cpm,max_cpm,hour_0,hour_1,hour_2,hour_3,hour_4,...,hour_22,hour_23,avg_ads_per_session,max_ads_per_session,sex,age,city_id,sex_binary,age_group,city_group
0,0,2,2053.830000,2053.830,3302.01,0,0,0,0,0,...,0,0,1.000000,1,2,19,0,0,19-35,0
1,1,82,124.068049,90.000,1140.00,0,0,0,0,0,...,0,0,2.157895,7,1,0,1,1,0-18,other
2,3,8,370.880000,288.130,715.92,0,0,0,0,0,...,0,0,1.000000,1,1,20,3,1,19-35,3
3,4,132,44.627955,30.160,255.36,0,0,0,0,0,...,0,0,2.095238,7,2,29,4,0,19-35,other
4,5,9,91.055556,42.500,390.00,0,0,0,0,0,...,0,0,1.285714,2,2,22,5,0,19-35,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25531,27764,43,210.409767,170.000,958.11,0,0,0,0,0,...,0,0,1.482759,3,1,38,295,1,36-50,other
25532,27765,79,146.901013,110.640,608.48,0,0,0,0,0,...,0,0,1.837209,7,2,30,79,0,19-35,other
25533,27766,54,159.045741,123.525,755.00,0,0,0,0,0,...,0,0,1.687500,6,2,21,1953,0,19-35,other
25534,27767,1,143.880000,143.880,143.88,0,0,0,0,0,...,0,0,1.000000,1,2,17,0,0,0-18,0


In [281]:
# Посмотрим информацию по датасету
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25536 entries, 0 to 25535
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   user_id              25536 non-null  int64   
 1   total_ads            25536 non-null  int64   
 2   avg_cpm              25536 non-null  float64 
 3   median_cpm           25536 non-null  float64 
 4   max_cpm              25536 non-null  float64 
 5   hour_0               25536 non-null  int64   
 6   hour_1               25536 non-null  int64   
 7   hour_2               25536 non-null  int64   
 8   hour_3               25536 non-null  int64   
 9   hour_4               25536 non-null  int64   
 10  hour_5               25536 non-null  int64   
 11  hour_6               25536 non-null  int64   
 12  hour_7               25536 non-null  int64   
 13  hour_8               25536 non-null  int64   
 14  hour_9               25536 non-null  int64   
 15  hour_10            

### 6. Создание целевых переменных

Вот несколько условий по задаче:
- Если ставка объявления `cpm` больше всех ставок всех остальных участников: со 100% вероятность выигрывает объявление со ставкой `cpm`
- Если ставка объявления `cpm` равна максимальной ставке среди всех остальных участников: объявление со ставкой `cpm` выигрывает с вероятностью в 50%. В реальном аукционе объявление может выигрывать по совершенно другим правилам и вероятностям, это упрощение для моделирования конкретно в данном датасете. 

Нам необходимо выявить веротяность выигрыша для каждой ставки

In [282]:
# Сортируем данные для упрощения обработки сессий и вычислений
history = history.sort_values(by=['user_id', 'hour'])

# Определяем max_cpm среди других участников на той же площадке и в тот же час
history['max_cpm_other'] = history.groupby(['hour', 'publisher'])['cpm'].transform(
    lambda x: x.nlargest(2).iloc[-1] if len(x) > 1 else x.iloc[0]
)

In [283]:
history

,hour,cpm,publisher,user_id,time_diff,new_session,session_id,max_cpm_other
778194,1186,805.65,1,0,0.0,0,0,1317.78
793167,1217,3302.01,1,0,31.0,1,1,1563.03
305,8,165.00,1,1,0.0,0,0,1660.56
1015,9,53.20,1,1,1.0,0,0,896.60
3458,9,36.00,1,1,0.0,0,0,896.60
...,...,...,...,...,...,...,...,...
782782,1188,285.00,2,27768,46.0,1,17,1319.16
790311,1188,205.00,12,27768,0.0,0,17,51.60
1059564,1216,210.00,2,27768,28.0,1,18,1568.13
936546,1410,179.80,2,27768,194.0,1,19,1123.18


In [284]:
# Вычисляем вероятность выигрыша в аукционе
history['win_probability'] = np.where(
    history['cpm'] > history['max_cpm_other'], 1,  # Победа с вероятностью 1, если ставка больше всех остальных
    np.where(history['cpm'] == history['max_cpm_other'], 0.5, 0)  # Ничья — вероятность 0.5, иначе 0
)

In [285]:
# Формируем уникальные просмотры с учетом правила "не показывать повторно" и сессий (разрыв > 6 часов)
def calculate_unique_views_with_probability(group):
    seen_ads = set()
    unique_views = []
    last_seen_time = {}
    for _, row in group.iterrows():
        user_id = row['user_id']
        if user_id in seen_ads:
            # Учет новой сессии
            if row['hour'] - last_seen_time[user_id] > 6:
                seen_ads.remove(user_id)
            else:
                unique_views.append(0)
                continue
        # Вероятностный учет просмотра
        unique_views.append(row['win_probability'])
        if row['win_probability'] > 0:
            seen_ads.add(user_id)
            last_seen_time[user_id] = row['hour']
    return unique_views

In [286]:
# Применяем к каждому пользователю
history['unique_view'] = (
    history.groupby('user_id', group_keys=False).apply(
        lambda group: pd.Series(calculate_unique_views_with_probability(group))
    ).values  # Конвертируем результат в массив для корректного присоединения
)

/var/folders/01/wgcp1v9n5zjdx14pvmwxt4mc0000gn/T/ipykernel_4204/4162637202.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  history.groupby('user_id', group_keys=False).apply(


In [287]:
history['unique_view'] = history['unique_view'].astype(float)  # Приводим уникальные просмотры к float

In [288]:
history

,hour,cpm,publisher,user_id,time_diff,new_session,session_id,max_cpm_other,win_probability,unique_view
778194,1186,805.65,1,0,0.0,0,0,1317.78,0.0,0.0
793167,1217,3302.01,1,0,31.0,1,1,1563.03,1.0,1.0
305,8,165.00,1,1,0.0,0,0,1660.56,0.0,0.0
1015,9,53.20,1,1,1.0,0,0,896.60,0.0,0.0
3458,9,36.00,1,1,0.0,0,0,896.60,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
782782,1188,285.00,2,27768,46.0,1,17,1319.16,0.0,0.0
790311,1188,205.00,12,27768,0.0,0,17,51.60,1.0,1.0
1059564,1216,210.00,2,27768,28.0,1,18,1568.13,0.0,0.0
936546,1410,179.80,2,27768,194.0,1,19,1123.18,0.0,0.0


In [289]:
# Вычисляем целевые переменные
def calculate_view_targets(group):
    cumsum = group['unique_view'].cumsum()
    return {
        'at_least_one': (cumsum >= 1).mean(),
        'at_least_two': (cumsum >= 2).mean(),
        'at_least_three': (cumsum >= 3).mean()
    }

# Группируем данные по пользователю и применяем подсчет целевых переменных
view_targets = (
    history.groupby('user_id').apply(calculate_view_targets).apply(pd.Series).reset_index()
)

# Переименовываем столбцы для удобства
view_targets.columns = ['user_id', 'at_least_one', 'at_least_two', 'at_least_three']

view_targets

/var/folders/01/wgcp1v9n5zjdx14pvmwxt4mc0000gn/T/ipykernel_4204/3308735392.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  history.groupby('user_id').apply(calculate_view_targets).apply(pd.Series).reset_index()


,user_id,at_least_one,at_least_two,at_least_three
0,0,0.500000,0.000000,0.000000
1,1,0.000000,0.000000,0.000000
2,3,0.000000,0.000000,0.000000
3,4,0.000000,0.000000,0.000000
4,5,0.000000,0.000000,0.000000
...,...,...,...,...
25531,27764,0.000000,0.000000,0.000000
25532,27765,0.379747,0.000000,0.000000
25533,27766,0.000000,0.000000,0.000000
25534,27767,0.000000,0.000000,0.000000


In [290]:
view_targets.describe()

,user_id,at_least_one,at_least_two,at_least_three
count,25536.000000,25536.000000,25536.000000,25536.000000
mean,13902.846217,0.180867,0.070898,0.039239
std,8018.785645,0.333918,0.206261,0.151234
min,0.000000,0.000000,0.000000,0.000000
25%,6953.750000,0.000000,0.000000,0.000000
50%,13934.500000,0.000000,0.000000,0.000000
75%,20849.250000,0.166667,0.000000,0.000000
max,27768.000000,1.000000,0.994536,0.978516


In [302]:
validate_answers.describe()

,at_least_one,at_least_two,at_least_three
count,1008.000000,1008.000000,1008.000000
mean,0.115441,0.065805,0.047170
std,0.146146,0.117812,0.099029
min,0.000000,0.000000,0.000000
25%,0.016650,0.000000,0.000000
50%,0.055250,0.009800,0.001400
75%,0.158700,0.078125,0.048950
max,0.930700,0.909700,0.883400


In [291]:
# Объединение целевых признаков с user_features
user_features = user_features.merge(view_targets, on='user_id', how='left')

In [300]:
pd.set_option('display.max_columns', None)

In [301]:
user_features

,user_id,total_ads,avg_cpm,median_cpm,max_cpm,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,avg_ads_per_session,max_ads_per_session,sex,age,city_id,sex_binary,age_group,city_group,at_least_one,at_least_two,at_least_three
0,0,2,2053.830000,2053.830,3302.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,1,2,19,0,0,19-35,0,0.500000,0.000000,0.000000
1,1,82,124.068049,90.000,1140.00,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.157895,7,1,0,1,1,0-18,other,0.000000,0.000000,0.000000
2,3,8,370.880000,288.130,715.92,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,1,1,20,3,1,19-35,3,0.000000,0.000000,0.000000
3,4,132,44.627955,30.160,255.36,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,2.095238,7,2,29,4,0,19-35,other,0.000000,0.000000,0.000000
4,5,9,91.055556,42.500,390.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.285714,2,2,22,5,0,19-35,other,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25531,27764,43,210.409767,170.000,958.11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.482759,3,1,38,295,1,36-50,other,0.000000,0.000000,0.000000
25532,27765,79,146.901013,110.640,608.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.837209,7,2,30,79,0,19-35,other,0.379747,0.000000,0.000000
25533,27766,54,159.045741,123.525,755.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.687500,6,2,21,1953,0,19-35,other,0.000000,0.000000,0.000000
25534,27767,1,143.880000,143.880,143.88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,1,2,17,0,0,0-18,0,0.000000,0.000000,0.000000


In [299]:
user_features.to_csv('../data/user_features.csv')

### Описание столбцов

| **Название столбца**     | **Описание**                                                                 |
|---------------------------|-----------------------------------------------------------------------------|
| `user_id`                | Уникальный идентификатор пользователя.                                      |
| `total_ads`              | Общее количество объявлений, просмотренных пользователем.                  |
| `avg_cpm`                | Средняя стоимость показов рекламы (`cpm`) для пользователя.                |
| `median_cpm`             | Медианная стоимость показов рекламы (`cpm`) для пользователя.              |
| `max_cpm`                | Максимальная стоимость показов рекламы (`cpm`) для пользователя.           |
| `hour_0`, ..., `hour_23` | Бинарные значения, показывающие, видел ли пользователь рекламу в данном часу. |
| `avg_ads_per_session`    | Среднее количество объявлений, просмотренных в одной сессии.               |
| `max_ads_per_session`    | Максимальное количество объявлений, просмотренных в одной сессии.          |
| `sex`                    | Пол пользователя (1 — мужской, 2 — женский).                               |
| `age`                    | Возраст пользователя.                                                      |
| `city_id`                | Уникальный идентификатор города проживания пользователя.                   |
| `sex_binary`             | Бинарное представление пола (1 — мужской, 0 — женский).                    |
| `age_group`              | Группа возраста пользователя (`0-18`, `19-35`, `36-50`, `51+`).            |
| `city_group`             | Группа города (топ-10 или `other`).                                        |
| `at_least_one`           | Доля показов, где пользователь видел объявление хотя бы один раз.          |
| `at_least_two`           | Доля показов, где пользователь видел объявление хотя бы два раза.          |
| `at_least_three`         | Доля показов, где пользователь видел объявление хотя бы три раза.          |

### Дополнительно:
- Столбцы `hour_0`, ..., `hour_23` не содержат количество показов, а лишь указывают, был ли пользователь активен в соответствующий час.
- Целевые переменные `at_least_one`, `at_least_two`, `at_least_three` отражают вероятность, рассчитанную по уникальным просмотрам.